In [1]:
import pandas as pd
import numpy as np
import warnings
import matplotlib.pyplot as plt

warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('soccer17.csv')
df = df.loc[(df.Y == 17)]
flds = ['Div','Date','Y','HomeTeam','AwayTeam','FTHG','FTAG']
df = df.loc[:,flds].copy()
df = df.rename({'HomeTeam':'Team_Home','AwayTeam':'Team_Away','FTHG':'G_Home','FTAG':'G_Away'}, axis = 1)
df.loc[:,'GameID'] = df.index
df['pd_Home'] = df.G_Home - df.G_Away
df['pd_Away'] = df.G_Away - df.G_Home

In [3]:
df_melt = pd.melt(df, id_vars = ['GameID', 'Div'], value_vars = ['Team_Home','Team_Away'], 
            var_name = 'isHome', value_name = 'Team')
df_melt.loc[:,'isHome'] = 1*(df_melt.isHome == 'Team_Home')

In [4]:
df_melt2 = pd.melt(df, id_vars=['GameID', 'Div'], value_vars=['pd_Home', 'pd_Away'], var_name = 'isHome', value_name = 'pd')
df_melt2.loc[:,'isHome'] = 1*(df_melt2.isHome == 'pd_Home')

In [5]:
df_merge = df_melt.merge(df_melt2, on=['GameID', 'Div', 'isHome'])
df_merge['Win'] = 1*(df_merge.pd > 0)
df_merge['Draw'] = 1*(df_merge.pd == 0)
df_merge['Loss'] = 1*(df_merge.pd < 0)

In [6]:
avg_pd = df_merge.groupby(['Team']).mean()['pd']
gp = df_merge.groupby(['Team']).size()
wins = df_merge.groupby(['Team']).agg({'Win': sum})['Win']
draws = df_merge.groupby(['Team']).agg({'Draw': sum})['Draw']
losses = df_merge.groupby(['Team']).agg({'Loss': sum})['Loss']
team = avg_pd.index
div = df_merge[['Div', 'Team']].drop_duplicates().sort_values('Team')['Div']

output = pd.DataFrame(list(zip(div, team, avg_pd, gp, wins, losses, draws)), 
                      columns=['Div', 'Team', 'Avg PD', 'Games Played', 'Wins', 'Losses', 'Draws'])

# 1a

In [7]:
df1 = output.loc[output.Div == 'EPL'].drop('Div', 1)
df1.sort_values(by='Avg PD', ascending=False).reset_index().drop('index', 1)

,Team,Avg PD,Games Played,Wins,Losses,Draws
0,Man City,2.078947,38,32,2,4
1,Liverpool,1.210526,38,21,5,12
2,Man United,1.052632,38,25,7,6
3,Tottenham,1.000000,38,23,7,8
4,Chelsea,0.631579,38,21,10,7
5,Arsenal,0.605263,38,19,13,6
6,Burnley,-0.078947,38,14,12,12
7,Leicester,-0.105263,38,12,15,11
8,Newcastle,-0.210526,38,12,18,8
9,Crystal Palace,-0.263158,38,11,16,11


# 1b

In [8]:
df1['Points'] = df1.Wins*3 + df1.Draws
df1.sort_values(by='Points', ascending=False).reset_index().drop('index', 1)

,Team,Avg PD,Games Played,Wins,Losses,Draws,Points
0,Man City,2.078947,38,32,2,4,100
1,Man United,1.052632,38,25,7,6,81
2,Tottenham,1.000000,38,23,7,8,77
3,Liverpool,1.210526,38,21,5,12,75
4,Chelsea,0.631579,38,21,10,7,70
5,Arsenal,0.605263,38,19,13,6,63
6,Burnley,-0.078947,38,14,12,12,54
7,Everton,-0.368421,38,13,15,10,49
8,Leicester,-0.105263,38,12,15,11,47
9,Crystal Palace,-0.263158,38,11,16,11,44


# 1c


In [9]:
df2 = output.sort_values(['Div', 'Avg PD'], ascending=[True, False])
df2.groupby('Div').head(3).reset_index(drop=1)

,Div,Team,Avg PD,Games Played,Wins,Losses,Draws
0,Bundesliga,Bayern Munich,1.882353,34,27,4,3
1,Bundesliga,Hoffenheim,0.529412,34,15,9,10
2,Bundesliga,Dortmund,0.500000,34,15,9,10
3,EPL,Man City,2.078947,38,32,2,4
4,EPL,Liverpool,1.210526,38,21,5,12
5,EPL,Man United,1.052632,38,25,7,6
6,La_Liga,Barcelona,1.842105,38,28,1,9
7,La_Liga,Real Madrid,1.315789,38,22,6,10
8,La_Liga,Ath Madrid,0.947368,38,23,5,10
9,Ligue_1,Paris SG,2.078947,38,29,3,6
